<a href="https://colab.research.google.com/github/MPrazzoli/corpsol/blob/main/Script_buono.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
pip install QuantLib

     |████████████████████████████████| 20.1MB 1.4MB/s 


In [ ]:
import os
from math import *
import numpy as np
import pandas as pd
from QuantLib import *
from datetime import date, datetime

path = '/content/drive/MyDrive/data.xlsx'

***Inserimento degli Input***

In [ ]:
try:
  underlying = input('Inserire il sottostante: ')
  if underlying == '':
    underlying = 'EUR3M'
except:
  underlying = 'EUR3M'

try:
  amortization_type = input('Inserire il piano di ammortamento: ')
  if amortization_type == '':
    amortization_type = 'Bullet'
except:
  amortization_type = 'Bullet'

try:
  notional = float(input('Inserire il nozionale: '))
except:
  notional = 100000.0

try:
  amortization_rate = float(input('Inserire il tasso annuo: '))
except:
  amortization_rate = .05

try:
  maturity_label = input('Inserire la durata del finanziamento in label ("M" per i mesi e "Y" per gli anni): ')
  if maturity_label == '':
     maturity_label = '5Y'
except:
  maturity_label = '5Y'

try:
  ref_date = Date(input('Inserire la data iniziale del finanziamento in formato dd/mm/yyyy: '),'%d/%m/%Y')
  if ref_date == '':
    ref_date = Date(date.today().day,date.today().month,date.today().year)
except:
  ref_date = Date(date.today().day,date.today().month,date.today().year)

try:
  tenor = int(input('Inserire il periodo che intercorre tra le rate in numero di mesi: '))
except:
  tenor = 3

try:
  business_convention = input('Inserire la convenzione:')
  if business_convention == '' or business_convention == 'ModifiedFollowing':
    business_convention = ModifiedFollowing
except:
  business_convention = ModifiedFollowing

try:
  date_gen_rule = input('Inserire la regola per il calcolo delle date:')
  if date_gen_rule == '' or date_gen_rule == 'Forward':
    date_gen_rule = DateGeneration.Forward
  else: 
    date_gen_rule = DateGeneration.Backward
except:
  date_gen_rule = DateGeneration.Forward

Inserire il sottostante: 
Inserire il piano di ammortamento: 
Inserire il nozionale: 
Inserire il tasso annuo: 
Inserire la durata del finanziamento in label ("M" per i mesi e "Y" per gli anni): 
Inserire la data iniziale del finanziamento in formato dd/mm/yyyy: 
Inserire il periodo che intercorre tra le rate in numero di mesi: 
Inserire la convenzione:
Inserire la regola per il calcolo delle date:


***Schedule Date***

In [ ]:
calendar = TARGET()
start_date = calendar.advance(ref_date,2,Days)
if maturity_label[-1] == 'M':
    maturity_date = calendar.advance(start_date, int(maturity_label.rsplit(maturity_label[-1], 1)[0]), Months)
elif maturity_label[-1] == 'Y':
    maturity_date = calendar.advance(start_date, int(maturity_label.rsplit(maturity_label[-1], 1)[0]), Years)
Tenor = Period(tenor, Months)
endofmonth = False
terminationDateConvention = business_convention
df = pd.DataFrame({'startdate':list(Schedule(start_date,maturity_date,Tenor,calendar,business_convention,terminationDateConvention,date_gen_rule,endofmonth))[:-1],'enddate':list(Schedule(start_date,maturity_date,Tenor,calendar,business_convention,terminationDateConvention,date_gen_rule,endofmonth))[1:]})

In [ ]:
# Calcolo numero delle rate totali dell'ammortamento
if maturity_label[-1] == 'M':
  rate_totali = int(12/tenor)
elif maturity_label[-1] == 'Y':
  rate_totali = int((12/tenor) * int(maturity_label.rsplit(maturity_label[-1], 1)[0]))

***Ammortamento Francese***

In [ ]:
def ammortamento_francese(amortization_rate,tenor,rate_totali,notional):
  
  t = (1 + amortization_rate / (12 / tenor)) ** rate_totali
  rata_fra = round(notional * t * (amortization_rate / (12 / tenor)) / (t - 1),2)
  Residuo_fra = notional
  Estinto_fra = 0
  tot_I_fra = 0
  plan_fra = np.array
  plan_fra = [[0,0,0,0,0, Residuo_fra]]
  rate_fra = range(0, rate_totali)
  for count in rate_fra:
    Int_quota_fra = round(Residuo_fra * (amortization_rate*(tenor/12)),2)
    Cap_quota_fra = (rata_fra - Int_quota_fra)
    tot_I_fra = (tot_I_fra + Int_quota_fra)
    Residuo_fra = (Residuo_fra - Cap_quota_fra)
    Estinto_fra = (Estinto_fra + Cap_quota_fra)
    Tot_fra = (Int_quota_fra + Cap_quota_fra)
    plan.append([Int_quota_fra, Cap_quota_fra, Tot_fra, tot_I_fra, Estinto_fra, Residuo_fra])
  index = ['Quota Interesse','Quota Capitale','Rata','Interessi Totali','Debito Estinto','Debito Residuo']
  df_fra = pd.DataFrame(plan_fra)
  df_fra.columns = index
  df1_fra = pd.concat([df,df_fra], axis=1)
  df2_fra = pd.concat([df,df_fra['Debito Residuo']], axis=1)[:-1]

  return (df2_fra)

***Ammortamento Italiano***


In [ ]:
def ammortamento_italiano(amortization_rate,tenor,rate_totali, notional):

  Residuo_ita = notional
  tot_I_ita = 0
  Estinto_ita = 0
  Tot_ita = 0
  plan_ita = np.array
  plan_ita = [[0,0,0,0,0,Residuo_ita]]
  rate_ita = range(0,rate_totali)
  for count in rate_ita:
    Int_quota_ita = round(Residuo_ita*amortization_rate/(12/tenor),2)
    Cap_quota_ita = round((notional/rate_totali),2)
    tot_I_ita = (tot_I_ita + Int_quota_ita)
    Residuo_ita = (Residuo_ita - Cap_quota_ita)
    Estinto_ita = (Estinto_ita + Cap_quota_ita)
    Tot_ita = (Int_quota_ita + Cap_quota_ita)
    plan_ita.append([Int_quota_ita,Cap_quota_ita,Tot_ita,tot_I_ita,Estinto_ita,Residuo_ita])
  index = ['Quota Interesse','Quota Capitale','Rata','Interessi Totali','Debito Estinto','Debito Residuo']
  df_ita = pd.DataFrame(plan_ita)
  df_ita.columns=index
  df1_ita = pd.concat([df,df_ita],axis=1)
  df2_ita = pd.concat([df,df_ita['Debito Residuo']],axis=1)[:-1]
  
  return (df2_ita)
  

***Ammortamento Bullet***


In [ ]:
def ammortamento_bullet(amortization_rate,tenor,rate_totali, notional):

  Residuo_bullet = notional
  tot_I_bullet = 0
  plan_bullet = np.array
  plan_bullet = [[0,0,Residuo_bullet]]
  rate_bullet = range(0,rate_totali)
  for count in rate_bullet:
    Int_quota_bullet = round(Residuo_bullet*amortization_rate/(12/tenor),2)
    tot_I_bullet = (tot_I_bullet+Int_quota_bullet)
    plan_bullet.append([Int_quota_bullet,tot_I_bullet,Residuo_bullet])
  index = ['Quota Interesse','Interessi Totali','Debito Residuo']
  df_bullet = pd.DataFrame(plan_bullet)
  df_bullet.columns = index
  df1_bullet = pd.concat([df,df_bullet],axis=1)
  df2_bullet = pd.concat([df,df_bullet['Debito Residuo']],axis=1)
  df2_bullet = df2_bullet[:-1]
  
  return (df2_bullet)

***Output del Modello***


In [ ]:
if (amortization_type == 'Francese'):
  df_output = (ammortamento_francese(amortization_rate,tenor,rate_totali,notional))
elif (amortization_type == 'Italiano'):
  df_output = (ammortamento_italiano(amortization_rate,tenor,rate_totali,notional))
elif (amortization_type == 'Bullet'):
  df_output = (ammortamento_bullet(amortization_rate,tenor,rate_totali,notional))
  
print (df_output)

             startdate             enddate  Debito Residuo
0        May 4th, 2021    August 4th, 2021        100000.0
1     August 4th, 2021  November 4th, 2021        100000.0
2   November 4th, 2021  February 4th, 2022        100000.0
3   February 4th, 2022       May 4th, 2022        100000.0
4        May 4th, 2022    August 4th, 2022        100000.0
5     August 4th, 2022  November 4th, 2022        100000.0
6   November 4th, 2022  February 6th, 2023        100000.0
7   February 6th, 2023       May 4th, 2023        100000.0
8        May 4th, 2023    August 4th, 2023        100000.0
9     August 4th, 2023  November 6th, 2023        100000.0
10  November 6th, 2023  February 5th, 2024        100000.0
11  February 5th, 2024       May 6th, 2024        100000.0
12       May 6th, 2024    August 5th, 2024        100000.0
13    August 5th, 2024  November 4th, 2024        100000.0
14  November 4th, 2024  February 4th, 2025        100000.0
15  February 4th, 2025       May 5th, 2025        100000

***Calcolo delle Fixing Date***

In [ ]:
try:
  rule_fixing = input('Inserire la regola per le fixing date: ')
  if rule_fixing == '':
     rule_fixing = '-2BD'
except:
  rule_fixing = '-2BD'

Inserire la regola per le fixing date: 


In [ ]:
column_list = ['fixing date','fixing start','fixing end']
for col in column_list:
  df_output[col] = np.zeros

for i, date in enumerate(df_output['startdate']):
  if rule_fixing == '-2BD':
    date_fix = calendar.advance(date, -2, Days)
  else:
    date_fix = calendar.advance(date, -1, Months)
    date_fix = calendar.endOfMonth(date_fix)
    date_fix = calendar.advance(date_fix, -1, Days)

  df_output['fixing date'].iloc[i] = date_fix
  df_output['fixing start'].iloc[i] = calendar.advance(date_fix, 2, Days)
for t, date in enumerate(df_output['fixing start']):
  df_output['fixing end'].iloc[t] = calendar.advance(date, tenor, Months, ModifiedFollowing)


In [ ]:
df_output

,startdate,enddate,Debito Residuo,fixing date,fixing start,fixing end
0,"May 4th, 2021","August 4th, 2021",100000.0,"April 30th, 2021","May 4th, 2021","August 4th, 2021"
1,"August 4th, 2021","November 4th, 2021",100000.0,"August 2nd, 2021","August 4th, 2021","November 4th, 2021"
2,"November 4th, 2021","February 4th, 2022",100000.0,"November 2nd, 2021","November 4th, 2021","February 4th, 2022"
3,"February 4th, 2022","May 4th, 2022",100000.0,"February 2nd, 2022","February 4th, 2022","May 4th, 2022"
4,"May 4th, 2022","August 4th, 2022",100000.0,"May 2nd, 2022","May 4th, 2022","August 4th, 2022"
5,"August 4th, 2022","November 4th, 2022",100000.0,"August 2nd, 2022","August 4th, 2022","November 4th, 2022"
6,"November 4th, 2022","February 6th, 2023",100000.0,"November 2nd, 2022","November 4th, 2022","February 6th, 2023"
7,"February 6th, 2023","May 4th, 2023",100000.0,"February 2nd, 2023","February 6th, 2023","May 8th, 2023"
8,"May 4th, 2023","August 4th, 2023",100000.0,"May 2nd, 2023","May 4th, 2023","August 4th, 2023"
9,"August 4th, 2023","November 6th, 2023",100000.0,"August 2nd, 2023","August 4th, 2023","November 6th, 2023"


Npv Obiettivo: prova

In [ ]:
df_output['differenza_giorni'] = (df_output['enddate'] - df_output['startdate']) / 360
df_output['prod'] = (df_output['Debito Residuo'] * df_output['differenza_giorni']) * 0.0001
BPV = df_output['prod'].sum()

*Class: Market Data*

In [ ]:
class historical_fixed_rate(object):

    def __init__(self, name, reference_date = None, currency = None, underlying = None):
        
        try:
            self.reference_date = datetime.strptime(reference_date, '%d/%m/%Y')
        except:
            self.reference_date = None
        
        self.name = name
        self.currency = currency
        self.underlying = underlying
        self.historical_fixed_rate_df = pd.DataFrame

class zero_curve_rate(object):
# chiedere a Mirko l'assegnazione di interp_rule e day_count
    def __init__(self, name, reference_date = None, currency = None, underlying = None, interp_rule = None, day_count = None):
        
        self.name = name
        
        try:
            self.reference_date = datetime.strptime(reference_date, '%d/%m/%Y')
        except:
            self.reference_date = None
      
        self.currency = currency
        self.underlying = underlying
        self.interpolation_rule = interp_rule
        self.day_count = day_count
        self.zero_curve_rate_df = pd.DataFrame

In [ ]:
def market_data(path):

  historical_fixed_rates = historical_fixed_rate(name ="HISTORICAL RATE")
  historical_fixed_rates.historical_fixed_rate_df = pd.read_excel(path, sheet_name='fixings')

  estimation_curve = zero_curve_rate(name='ESTIMATION')
  estimation_curve.zero_curve_rate_df = pd.read_excel(path, sheet_name='estimation')

  discount_curve = zero_curve_rate(name='DISCOUNT')
  discount_curve.zero_curve_rate_df = pd.read_excel(path, sheet_name='discount')

  market_data_dict = {'ESTIMATION': estimation_curve, 'DISCOUNT': discount_curve, 'HISTORICAL': historical_fixed_rates}

  return(market_data_dict)

In [ ]:
# converto le date della curva importata in Date quantlib
market_dict = market_data(path)
for i, d in enumerate(market_dict['ESTIMATION'].zero_curve_rate_df['DATE']):
  market_dict['ESTIMATION'].zero_curve_rate_df['DATE'].iloc[i] = Date(d.day,d.month,d.year)

In [ ]:
# funzione per convertire le Date quantlib in numeri ATTENZIONE RESTITUISCE SEMPRE UNA LISTA!!!
def date_to_ordinal(date_to_convert):
  date_number = []
  if type(date_to_convert) == QuantLib.Date:
    date_convert = datetime(date_to_convert.year(),date_to_convert.month(),date_to_convert.dayOfMonth())
    date_number.append(date_convert.toordinal())
  else:
    for i, d in enumerate(date_to_convert):
      date_convert = datetime(d.year(), d.month(), d.dayOfMonth())
      date_number.append(date_convert.toordinal())
  return date_number


In [ ]:
def rate_interpoletor(interpolation_type, x_interpolation, y_interpolation, date_to_interpolate):
    interpolation_object = interpolation_type(x_interpolation, y_interpolation)
    rate_interpolated = interpolation_object(date_to_interpolate, allowExtrapolation=True)
    return rate_interpolated

ESTIMATION RATE FUNCTION

In [ ]:
def estimation_rate_function(dataframe_curve, date_fixing_start, date_fixing_end, ref_date): # da passare i fixed rate !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
  date_curve_number = date_to_ordinal( dataframe_curve['DATE'])      
  date_fixing_start_number = date_to_ordinal(date_fixing_start)
  date_fixing_end_number = date_to_ordinal(date_fixing_end)
  ref_date_number = date_to_ordinal(ref_date)
  rate_start = []
  for date in date_fixing_start_number:
    rate = rate_interpoletor(KrugerCubic, list(date_curve_number), list(dataframe_curve['RATE']), date)
    rate_start.append(rate)
  discount_start = []
  for i, rate in enumerate(rate_start):
    discount_start.append(exp(-rate / 100 * ((date_fixing_start_number[i] - ref_date_number[0]) / 365))) # convenzione curva di estimation dell'EURIBOR è act/365
  rate_end = []
  for date in date_fixing_end_number:
    rate_end.append(rate_interpoletor(KrugerCubic, list(date_curve_number), list(dataframe_curve['RATE']), date))
  discount_end = []
  for i, rate in enumerate(rate_end):
    discount_end.append(exp(-rate / 100 * ((date_fixing_end_number[i] - ref_date_number[0]) / 365)))
  estimated_rate = []
  for i, date in enumerate(date_fixing_start_number):
    if date <= ref_date_number[0] : 
      estimated_rate.append(0) # bisogna mettere il rate che fa riferimento al sottostante !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
    else:
      estimated_rate.append((discount_start[i] / discount_end[i] - 1) / ((date_fixing_end_number[i] - date) / 360)) # convenzione per i discount dell'EURIBOR è act/360
  return estimated_rate

In [ ]:
a=estimation_rate_function(market_dict['ESTIMATION'].zero_curve_rate_df,df_output['fixing start'], df_output['fixing end'], ref_date)


In [ ]:
date_to_ordinal(df_output['startdate'][0])

[737914]

In [ ]:
dr = discount_rate(market_dict['DISCOUNT'].zero_curve_rate_df,a,ref_date,df_output)

In [ ]:
def discount_rate(dataframe_curve, estimated_rate, ref_date, dataframe_schedule):
  date_curve_number = date_to_ordinal(dataframe_curve['DATE'])
  end_date_number = date_to_ordinal(dataframe_schedule['enddate'])
  ref_date_number = date_to_ordinal(ref_date)
  settlement_date_number = date_to_ordinal(dataframe_schedule['startdate'][0])
  rate_list = []  #capire che rate sono
  for date in end_date_number:
    rate = rate_interpoletor(KrugerCubic, list(date_curve_number), estimated_rate, date)
    rate_list.append(rate)
  settlemnt_date_rate = rate_interpoletor(KrugerCubic, list(date_curve_number), estimated_rate, settlement_date_number[0])
  discount_list = []
  for i, rate_curve in enumerate(rate_list):
    discount = exp(-rate_curve * (end_date_number[i] - ref_date_number[0])/ 365)
    discount_list.append(discount)
  discount_settlement = exp(-settlemnt_date_rate * ((settlement_date_number[0] - ref_date_number[0]) / 365))
  discount_rates = []
  for discount in discount_list:
    discount_rates.append(discount / discount_settlement)
  return discount_rates

In [ ]:
df_output['disc'] = dr


In [ ]:
df_output

,startdate,enddate,Debito Residuo,fixing date,fixing start,fixing end,differenza_giorni,prod,disc
0,"May 4th, 2021","August 4th, 2021",100000.0,"April 30th, 2021","May 4th, 2021","August 4th, 2021",0.255556,2.55556,1.000013
1,"August 4th, 2021","November 4th, 2021",100000.0,"August 2nd, 2021","August 4th, 2021","November 4th, 2021",0.255556,2.55556,1.000023
2,"November 4th, 2021","February 4th, 2022",100000.0,"November 2nd, 2021","November 4th, 2021","February 4th, 2022",0.255556,2.55556,1.000029
3,"February 4th, 2022","May 4th, 2022",100000.0,"February 2nd, 2022","February 4th, 2022","May 4th, 2022",0.247222,2.47222,1.000031
4,"May 4th, 2022","August 4th, 2022",100000.0,"May 2nd, 2022","May 4th, 2022","August 4th, 2022",0.255556,2.55556,1.000014
5,"August 4th, 2022","November 4th, 2022",100000.0,"August 2nd, 2022","August 4th, 2022","November 4th, 2022",0.255556,2.55556,1.000044
6,"November 4th, 2022","February 6th, 2023",100000.0,"November 2nd, 2022","November 4th, 2022","February 6th, 2023",0.261111,2.61111,1.000100
7,"February 6th, 2023","May 4th, 2023",100000.0,"February 2nd, 2023","February 6th, 2023","May 8th, 2023",0.241667,2.41667,1.000114
8,"May 4th, 2023","August 4th, 2023",100000.0,"May 2nd, 2023","May 4th, 2023","August 4th, 2023",0.255556,2.55556,1.000129
9,"August 4th, 2023","November 6th, 2023",100000.0,"August 2nd, 2023","August 4th, 2023","November 6th, 2023",0.261111,2.61111,1.000143


In [ ]:
class IRS:

  def __init__(self):

    self.start_date = None
    self.maturity_date = None
    self.tenor = None
    self.business_convention = None
    self.date_gen_rule = None
    self.schedule = pd.DataFrame
    self.eom = None

    
    self.npv = None

In [ ]:
def pricer(path,
           underlying,
           amortization_type,
           notional,
           amortization_rate,
           maturity_label,
           ref_date,
           tenor,
           business_convention,
           date_gen_rule,
           currency = 'EUR',
           calendar = TARGET(),
           end_of_month = False,
           deduction_formula = '-1M LBD/-1BD',
           day_count = Actual360):

    dictionary = market_data(path)
    irs = IRS()
    irs.start_date = calendar.advance(ref_date,2,Days)
    if maturity_label[-1] == 'M':
      irs.maturity_date = calendar.advance(irs.start_date, int(maturity_label.rsplit(maturity_label[-1], 1)[0]), Months)
    elif maturity_label[-1] == 'Y':
      irs.maturity_date = calendar.advance(irs.start_date, int(maturity_label.rsplit(maturity_label[-1], 1)[0]), Years)
    irs.business_convention = business_convention
    irs.tenor = tenor
    irs.date_gen_rule = date_gen_rule
    irs.eom = end_of_month

# aggiungere altri attribuiti all'irs(calendario,EOM, deduction formula, day count)


    
    irs.schedule = pd.DataFrame({'startdate':list(Schedule(irs.start_date,irs.maturity_date,irs.tenor,calendar,irs.business_convention,irs.business_convention,irs.date_gen_rule,irs.eom))[:-1],'enddate':list(Schedule(irs.start_date,irs.maturity_date,irs.tenor,calendar,irs.business_convention,irs.business_convention,irs.date_gen_rule,irs.eom))[1:]})

In [ ]:
class IRS:

  def __init__(self, leg1_name = 'Receive', leg2_name = 'Pay', leg1_rate_type = 'Fixed', leg2_rate_type = 'Floating'):

    self.start_date = None
    self.leg1_flows = pd.DataFrame
    self.leg2_flows = pd.DataFrame
    self.leg1_name = leg1_name
    self.leg2_name = leg2_name
    self.leg1_rate_type = leg1_rate_type  
    self.leg2_rate_type = leg2_rate_type
    self.npv = None

In [ ]:
def submit(path,
           underlying,
           amortization_type,
           notional,
           amortization_rate,
           maturity_label,
           ref_date,
           tenor,
           business_convention,
           date_gen_rule,
           currency = 'EUR',
           calendar = TARGET(),
           end_of_month = False,
           deduction_formula = '-1M LBD/-1BD',
           day_count = Actual360):

  irs = IRS()

In [ ]:
submit(path = market_data_path, underlying = underlying, amortization_type = amortization_type, notional = notional, amortization_rate = amortization_rate, maturity_label = maturity_label, ref_date = ref_date, tenor = tenor,
       business_convention = business_convention, date_gen_rule = date_gen_rule)

In [ ]:
leg_fixed=FixedRateLeg(schedule,Actual360(),[notional],[0.05])
for i in leg_fixed:
  print(i.date(),i.amount())